## Hovmoller Sites

This notebook opens a shape file of transects, allows you to select a transect by number, and plot a hovmoller diagram of the site based on datacube landsat surface reflectance data and BoM rainfall data.

* if the time period is changed, you will need to re-save the data manually by changing 'savepath' so that files are not overwritten
* you will need to change coordinate reference system specs if the shapefile is not in Albers.
* rainfall data is currently in WGS84 due to datacube issues. This isn't a problem at the moment but could be later.
* if running for a time period where one of the sensors is not available, you may need to comment out statements referring to that sensor

Dependencies:

  * shapefile of the site transects in EPSG 3577/Australian Albers
  
  
The following sensors are available for the following time frames:
* Landsat 5 - 1986 to April 1999  followed by a gap until May 2003 - November 2011 (data from 2009 onwards becomes less reliable in southern Australia)
* Landsat 7 - April 1999 to present, however after May 2003 the scan line corrector (SLC) failed, 
so data are referred to as SLC-off, meaning they've got a venetian blinds appearance with wedges of missing data
  * This data is not well suited for inclusion in composites, but is fine to use in time series analysis
* Landsat 8 - April 2013 onwards
  
Bex Dunn april 2017

In [2]:
#in this notebook we want to plot non-interactively
%matplotlib inline

#get a bunch of modules to use

#suppress warnings thrown when rainfall data is imported
import logging
logging.getLogger('rasterio._gdal').setLevel(logging.ERROR)
import warnings

import datacube
import xarray as xr
from datacube.storage import masking
from datacube.storage.masking import mask_to_dict
import datetime as dt

import matplotlib as mpl
from matplotlib import colors
from matplotlib.backends.backend_pdf import PdfPages
from matplotlib import pyplot as plt
import matplotlib.dates
import json
import pandas as pd
from IPython.display import display
import ipywidgets as widgets

from shapely.geometry import shape
import matplotlib.gridspec as gridspec
import numpy as np
import pickle
import os
import geopandas as gpd

#libraries for polygon and polygon mask
import fiona
import shapely.geometry
import rasterio.features
from datacube.utils import geometry
from datacube.helpers import ga_pq_fuser as pq_fuser
from datacube.storage.masking import mask_valid_data as mask_invalid_data
import rasterio
#tells the datacube what app you are planning on using
dc = datacube.Datacube(app='dc-BOMRainfallandNbar')

### Set up functions for this notebook

In [3]:
#This defines the function that converts a linear vector file into a string of x,y coordinates
def geom_query(geom, geom_crs='EPSG:3577'):
    """
    Create datacube query snippet for geometry
    """
    return {
        'x': (geom.bounds[0], geom.bounds[2]),
        'y': (geom.bounds[1], geom.bounds[3]),
        'crs': geom_crs
    }


def warp_geometry(geom, crs_crs, dst_crs):
    """
    warp geometry from crs_crs to dst_crs
    """
    return shapely.geometry.shape(rasterio.warp.transform_geom(crs_crs, dst_crs, shapely.geometry.mapping(geom)))


def transect(data, geom, resolution, method='nearest', tolerance=None):
    """
    gets the transect
    """
    #Changed for py3 compatibility 17.03.17
    dist = [i for i in range(0, int(geom.length), resolution)]
    #points = zip(*[geom.interpolate(d).coords[0] for d in dist]) py2
    points = list(zip(*[geom.interpolate(d).coords[0] for d in dist])) #py3
    indexers = {
        data.crs.dimensions[0]: list(points[1]),
        data.crs.dimensions[1]: list(points[0])        
    }
    return data.sel_points(xr.DataArray(dist, name='distance', dims=['distance']),
                           method=method,
                           tolerance=tolerance,
                           **indexers)

#mask to use with our polygons to calculate wet and dry years 
#(comes from http://agdc-v2.readthedocs.io/en/stable/user/guide.html#code-recipes)
def geometry_mask(geoms, geobox, all_touched=False, invert=False):
    """
    Create a mask from shapes.

    By default, mask is intended for use as a
    numpy mask, where pixels that overlap shapes are False.
    :param list[Geometry] geoms: geometries to be rasterized
    :param datacube.utils.GeoBox geobox:
    :param bool all_touched: If True, all pixels touched by geometries will be burned in. If
                             false, only pixels whose center is within the polygon or that
                             are selected by Bresenham's line algorithm will be burned in.
    :param bool invert: If True, mask will be True for pixels that overlap shapes.
    """
    return rasterio.features.geometry_mask([geom.to_crs(geobox.crs) for geom in geoms],
                                           out_shape=geobox.shape,
                                           transform=geobox.affine,
                                           all_touched=all_touched,
                                           invert=invert)

## add the path to the input shapefile

In [4]:
#transects file will be the file containing our transects
transects_file = '/g/data1/r78/rjd547/groundwater_activities/GalileeBasin/shapefiles/landsat_transects_Galilee_BA.shp'
transects_df = gpd.read_file(transects_file)
#print(transects_file)

## Define the epoch

In [5]:
#Define temporal range - testing with one year of data
start_of_epoch = '1987-10-01'
#need a variable here that defines a rolling 'latest observation'  
end_of_epoch =  '2016-10-01'
print ('temporal range is '+start_of_epoch+' to '+end_of_epoch)

temporal range is 1987-10-01 to 2016-10-01


### Set up wavelengths/bands/sensors/TSS and brightness adjustments

In [6]:
#Define wavelengths/bands of interest, remove this kwarg to retrieve all bands
bands_of_interest = ['blue',
                     'green',
                     'red', 
                     'nir',
                     'swir1', 
                     'swir2'
                     ]

#Define sensors of interest
sensor1 = 'ls5'
sensor2 = 'ls7'
sensor3 = 'ls8'

#Group PQ by solar day to avoid idiosyncracies of N/S overlap differences in PQ algorithm performance
pq_albers_product = dc.index.products.get_by_name(sensor1+'_pq_albers')
valid_bit = pq_albers_product.measurements['pixelquality']['flags_definition']['contiguous']['bits']

#load sensor specific band adjustment tuples for TSS and brightness
brightness_coeff = {}
brightness_coeff['ls5'] = (0.304, 0.279, 0.434, 0.559, 0.508, 0.186)
brightness_coeff['ls7'] = (0.304, 0.279, 0.434, 0.559, 0.508, 0.186)
#brightness_coeff['ls7'] = (0.3561, 0.3972, 0.3904, 0.6966, 0.2286, 0.1596)
brightness_coeff['ls8'] = (0.3029, 0.2786, 0.4733, 0.5599, 0.508, 0.1872)

greenness_coeff = {}
greenness_coeff['ls5'] = (-0.285, -0.244, -0.544, 0.724, 0.084, -0.180)
greenness_coeff['ls7'] = (-0.285, -0.244, -0.544, 0.724, 0.084, -0.180)
#greenness_coeff['ls7'] = (-0.3344, -0.3544, -0.4556, 0.6966, -0.0242, -0.263)
greenness_coeff['ls8'] = (-0.2941, -0.243, -0.5424, 0.7276, 0.0713, -0.1608)

wetness_coeff = {}
wetness_coeff['ls5'] = (0.151, 0.179, 0.330, 0.341, -0.711, -0.457)
wetness_coeff['ls7'] = (0.151, 0.179, 0.330, 0.341, -0.711, -0.457)
#wetness_coeff['ls7'] = (0.2626, 0.2141, 0.0926, 0.0656, -0.7629, -0.5388)
wetness_coeff['ls8'] = (0.1511, 0.1973, 0.3283, 0.3407, -0.7117, -0.4559)

tsm_coeff = {
    'ls5': (3983, 1.6246),
    'ls7': (3983, 1.6246),
    'ls8': (3957, 1.6436)
}

In [7]:
# Have a look at the dataframe of transects below to decide which one to plot

In [8]:
transects_df

,name,descript,age,length,geometry
0,Doongmabulla Springs 1,Mouldy Crumpet to Joshua spring Nth Sth transect,None,0.0,LINESTRING (333358.6094532241 -2378083.7166778...
1,Doongmabulla Springs 3,"Moses to Wobbly springs transect, includes sma...",None,0.0,LINESTRING (333420.8867547495 -2377528.8540737...
2,Doongmabulla Springs 2,Moses Springs transect,None,0.0,LINESTRING (333741.60005989 -2378101.391789466...
3,Doongmabulla Springs 2,Mouldy Crumpet to House springs across Carmich...,None,0.0,LINESTRING (331363.4966704529 -2377794.2407379...
4,Doongmabulla Springs 4,House Springs - Carmichael River transect,None,0.0,LINESTRING (334086.5102908454 -2376836.1455913...
5,Clematis - Carmichael River,Follows Carmichael river from springs across c...,None,0.0,LINESTRING (331761.7087712438 -2374087.3671648...
6,Rewan Group - Carmichael River,Rewan Group subcrop area under Carmichael River,None,0.0,LINESTRING (340855.5085843539 -2378145.6934241...
7,mine lease - Carmichael river,Carmichael River transect through mine leases,None,0.0,LINESTRING (349388.9787330463 -2379960.5883129...
8,Carmichael River floodplain,Carmichael river crossection just downstream o...,None,0.0,LINESTRING (359123.2453407632 -2381555.6643693...
9,Doongmabulla Springs 5,Joshua Springs - Carmichael River transect,None,0.0,LINESTRING (333750.8490890979 -2374947.4431831...


### loop over transects, pull in rainfall and nbar from datacube and output hovmoller plots

In [9]:
#use fiona module to open the shape file
transects = fiona.open(transects_file)
while True:
    #this statement asks you to choose a shapefile. put in a number and press enter.
    i = int(input("choose the index of the transect you would like to plot:"))
    #for i in range(len(transects)):
    geom = shape(transects[i]['geometry'])
    query = {
        'time': (start_of_epoch, end_of_epoch),
    }
    query.update(geom_query(geom, geom_crs=transects.crs_wkt)) #comment this out if not using a polygon

    #use the transect dataframe to get transect metadata
    transect_index = transects_df.index[i]
    transect_name = transects_df['name'][i] 
    if transects_df['descript'][i] is not None:
        transect_description = transects_df['descript'][i]
    else:
        transect_description = ' '
    try:
        print(transect_name+' : '+transect_description)
    except TypeError as err:
        print("TypeError: {0}".format(err))
        print(transect_name)

    #create a filename for the transect
    shape_name = transect_name.split()
    shape_name ='_'.join(shape_name)
    shape_name=shape_name+'_ix'+str(transect_index)
    print(shape_name)

    #setup a save directory for our data
    savepath ='/g/data/r78/rjd547/groundwater_activities/GalileeBasin/pickledata/'+shape_name+'/'
    try:
            os.mkdir(savepath)
    except OSError as err:
            print("OS error: {0}".format(err))
    try:
        #try to get the rain data from a pickle file saved earlier 
        f = open(savepath+'Rainfall_data'+'.pkl', 'rb')
        rain = pickle.load(f)
        Studysite_rain = rain['Studysite_rain']
        print('loaded rainfall grids from file:'+savepath+'Rainfall_data.pkl')
        f.close()
    except FileNotFoundError:
        #Grab bom_rainfall_grids from the datacube
        print('loaded bom rainfall grids from datacube')
        Studysite_rain = dc.load(product = 'bom_rainfall_grids', **query)
        #make a dictionary of the data we want to save
        vars2pickle = {'Studysite_rain':Studysite_rain}
        f = open(savepath+'Rainfall_data'+'.pkl', 'wb')
        pickle.dump(vars2pickle,f) 
        print('saved rainfall data to file')
        #pickle.dump(vars2pickle,f,protocol = 2, fix_imports = True) #maintain compatibility with python 2
        f.close()   

    #resample xarray Dataset Studysite_rain by Annual'AS' to get yearly avg with year starting in october
    #Note that the resampling we did means that each year is labelled according to its first day  
    rain_sp = Studysite_rain.mean(dim = ('latitude','longitude'))
    month_sp = rain_sp.resample('MS', dim = 'time', how = 'mean')
    year_avg = Studysite_rain.resample('AS-OCT', dim='time', how='mean', keep_attrs=True)
    # Create a spatial average
    year_avg_sp = year_avg.mean(dim = ('latitude', 'longitude'))

    try:
        #try to get the NBAR and PQ data for sensor n from a pickle file saved earlier
        f = open(savepath+'NBARPQ_data'+'.pkl', 'rb')
        NBPQdata = pickle.load(f)
        sensor1_nbar = NBPQdata['sensor1_nbar']
        sensor1_pq = NBPQdata['sensor1_pq']
        sensor2_nbar = NBPQdata['sensor2_nbar']
        sensor2_pq = NBPQdata['sensor2_pq']
        sensor3_nbar = NBPQdata['sensor3_nbar']
        sensor3_pq = NBPQdata['sensor3_pq']
        print('loaded sensor NBAR and PQ from file')
        f.close()
    except FileNotFoundError:
        #Retrieve the NBAR and PQ data for sensor n from the datacube
        sensor1_nbar = dc.load(product= sensor1+'_nbar_albers', group_by='solar_day', measurements = bands_of_interest,  **query)
        sensor1_pq = dc.load(product= sensor1+'_pq_albers', group_by='solar_day', fuse_func=pq_fuser, **query)        
        print('loaded sensor1 NBAR and PQ from datacube')

        sensor2_nbar = dc.load(product= sensor2+'_nbar_albers', group_by='solar_day', measurements = bands_of_interest,  **query)
        sensor2_pq = dc.load(product= sensor2+'_pq_albers', group_by='solar_day', fuse_func=pq_fuser, **query)                  
        print('loaded sensor2 NBAR and PQ from datacube')

        sensor3_nbar = dc.load(product= sensor3+'_nbar_albers', group_by='solar_day', measurements = bands_of_interest,  **query)
        sensor3_pq = dc.load(product= sensor3+'_pq_albers', group_by='solar_day', fuse_func=pq_fuser, **query)                  
        print('loaded sensor3 NBAR and PQ from datacube')

        #make a dictionary of the data we want to save
        pickle_vars = {'sensor1_nbar':sensor1_nbar,'sensor1_pq':sensor1_pq,
                  'sensor2_nbar':sensor2_nbar,'sensor2_pq':sensor2_pq,
                  'sensor3_nbar':sensor3_nbar,'sensor3_pq':sensor3_pq}

        f = open(savepath+'NBARPQ_data'+'.pkl', 'wb')
        pickle.dump(pickle_vars,f) 
        print('saved sensor NBAR and PQ data to file')
        #pickle.dump(pickle_vars,f,protocol = 2, fix_imports = True) #maintain compatibility with python 2
        f.close()

        #save attributes
        crs = sensor1_nbar.crs
        crswkt = sensor1_nbar.crs.wkt
        affine = sensor1_nbar.affine

    s1_cloud_free = masking.make_mask(sensor1_pq, 
                                  cloud_acca='no_cloud',
                                  cloud_shadow_acca = 'no_cloud_shadow',
                                  cloud_shadow_fmask = 'no_cloud_shadow',
                                  cloud_fmask='no_cloud',
                                  blue_saturated = False,
                                  green_saturated = False,
                                  red_saturated = False,
                                  nir_saturated = False,
                                  swir1_saturated = False,
                                  swir2_saturated = False,
                                  contiguous=True)
    s1_good_data = s1_cloud_free.pixelquality.loc[start_of_epoch:end_of_epoch]
    sensor1_nbar = sensor1_nbar.where(s1_good_data)
    sensor1_nbar.attrs['crs'] = crs
    sensor1_nbar.attrs['affine'] = affine

    s2_cloud_free = masking.make_mask(sensor2_pq, 
                                  cloud_acca='no_cloud',
                                  cloud_shadow_acca = 'no_cloud_shadow',
                                  cloud_shadow_fmask = 'no_cloud_shadow',
                                  cloud_fmask='no_cloud',
                                  blue_saturated = False,
                                  green_saturated = False,
                                  red_saturated = False,
                                  nir_saturated = False,
                                  swir1_saturated = False,
                                  swir2_saturated = False,
                                  contiguous=True)
    s2_good_data = s2_cloud_free.pixelquality.loc[start_of_epoch:end_of_epoch]
    sensor2_nbar = sensor2_nbar.where(s2_good_data)
    sensor2_nbar.attrs['crs'] = crs
    sensor2_nbar.attrs['affine'] = affine

    #sensor 3 is commented out if the time period does not include landsat 8

    s3_cloud_free = masking.make_mask(sensor3_pq, 
                                  cloud_acca='no_cloud',
                                  cloud_shadow_acca = 'no_cloud_shadow',
                                  cloud_shadow_fmask = 'no_cloud_shadow',
                                  cloud_fmask='no_cloud',
                                  blue_saturated = False,
                                  green_saturated = False,
                                  red_saturated = False,
                                  nir_saturated = False,
                                  swir1_saturated = False,
                                  swir2_saturated = False,
                                  contiguous=True)
    s3_good_data = s3_cloud_free.pixelquality.loc[start_of_epoch:end_of_epoch]
    sensor3_nbar = sensor3_nbar.where(s3_good_data)
    sensor3_nbar.attrs['crs'] = crs
    sensor3_nbar.attrs['affine'] = affine

    #Set up some colour maps for the Hovmoller plots

    #This controls the colour maps used for plotting NDVI, TSS and Normalised Burn Ratio
    ndvi_cmap = mpl.colors.ListedColormap(['blue', '#ffcc66','#ffffcc' , '#ccff66' , '#2eb82e', '#009933' , '#006600'])
    tss_cmap = mpl.colors.ListedColormap(['navy', 'blue', 'deepskyblue','aquamarine' , 'burlywood' , 'saddlebrown'])
    #nbr_cmap = mpl.colors.ListedColormap(['darkslategray','dimgray' , 'lightgrey' , '#ccff66' , '#2eb82e', '#009933'])

    ndvi_bounds = [-1, 0, 0.1, 0.25, 0.35, 0.5, 0.8, 1]
    #nbr_bounds = [-1, -0.1, 0, 0.1, 0.2, 0.4, 0.6]
    tss_bounds = [0, 10, 20, 50, 80, 100, 150]

    tss_norm = mpl.colors.BoundaryNorm(tss_bounds, tss_cmap.N)
    ndvi_norm = mpl.colors.BoundaryNorm(ndvi_bounds, ndvi_cmap.N)
    #nbr_norm = mpl.colors.BoundaryNorm(nbr_bounds, nbr_cmap.N)

    #Retrieve the hovmoller data volumes and concatenate them

    geom_w = warp_geometry(geom, query['crs'], crs.wkt)
    #have set this rather than the specific exact form of the query..
    #warpy = {} #kludge 
    #warpy['crs'] = 'EPSG:3577'
    #geom_w = warp_geometry(geom, warpy['crs'], crs.wkt)
    hov_sensor1 = transect(sensor1_nbar, geom_w, 25)
    hov_sensor2 = transect(sensor2_nbar, geom_w, 25)
    hov_sensor3 = transect(sensor3_nbar, geom_w, 25)
    #--------
    #hov_multi = xr.concat([hov_sensor1, hov_sensor2], dim = 'time')
    #hov_multi = hov_sensor1 #FIXME
    hov_multi = xr.concat([hov_sensor1, hov_sensor2, hov_sensor3], dim = 'time')
    time_sorted = hov_multi.time.argsort()
    hov_multi = hov_multi.isel(time=time_sorted)

    #print ('The number of time slices unfiltered = '+ str(hov_multi.time.size))
    # Set the percentage of good data that you'd like to display with pernan variable - 0.9 will return rows that have 90%
    # of valid values
    pernan = 0.8
    hov_multi = hov_multi.dropna('time',  thresh = int(pernan*hov_multi.distance.size))
    #print ('The number of time slices filtered = '+str(hov_multi.time.size))

    #Use this number to flag results as water or land (to be replaced with WOfS once available)
    #this will throw a warning about nan comparison in xarray. 
    waterthresh = 500
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        hov_multi_land = hov_multi.where(hov_multi.swir1>waterthresh)
        hov_multi_water = hov_multi.where(hov_multi.swir1<waterthresh)

    hov_multi_ndvi = ((hov_multi.nir-hov_multi.red)/(hov_multi.nir+hov_multi.red))
    hov_multi_nbr = ((hov_multi.nir-hov_multi.swir2)/(hov_multi.nir+hov_multi.swir2))
    #NBR as defined in http://www.sciencedirect.com/science/article/pii/S0034425706005128
    hov_multi_ndwi = ((hov_multi.nir-hov_multi.swir1)/(hov_multi.nir+hov_multi.swir1))
    #NDWI as per http://www.sciencedirect.com/science/article/pii/S0034425703003353 and 
    #http://www.sciencedirect.com/science/article/pii/S0034425796000673

    hov_multi_tsm = (tsm_coeff[sensor1][0]*((hov_multi_water.green + hov_multi_water.red)/20000)**tsm_coeff[sensor1][1])
    #hov_multi_ndvi_drop = ((hov_multi_drop.nir-hov_multi.red)/(hov_multi_drop.nir+hov_multi.red))
    hov_multi_ndvi_drop = hov_multi_ndvi.dropna('time',  thresh = int(pernan*hov_multi.distance.size))
    
    
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        #try plotting the first clean time slice to see where we are
        print('transect image: ')
        rgb = sensor1_nbar.isel(time =0).to_array(dim='color').sel(color=['swir1', 'nir', 'green']).transpose('y', 'x', 'color')
        fake_saturation = 4500
        clipped_visible = rgb.where(rgb<fake_saturation).fillna(fake_saturation)
        max_val = clipped_visible.max(['y', 'x'])
        scaled = (clipped_visible / max_val)
        plt.imshow(scaled, interpolation = 'nearest',
                   extent=[scaled.coords['x'].min(), scaled.coords['x'].max(), 
                           scaled.coords['y'].min(), scaled.coords['y'].max()])
        ##turn this one on to see where the transect is on the image!
        #plt.scatter(x=hov_sensor1.coords['x'], y=hov_sensor1.coords['y'], c='r')
        plt.show()
    
    #Make a hovmoller plot and save it to file.
    #Use firstyear and last year to zoom into periods of interest
    firstyearhov = start_of_epoch#'1987-06-01'
    lastyearhov = end_of_epoch #'2016-12-31'
    fig = plt.figure(figsize=(11.27,11.69))
    gs = gridspec.GridSpec(1, 2, width_ratios=[1,3])
    ax1 = plt.subplot(gs[0, 0])
    ax1.plot(month_sp.rainfall, month_sp.time, color='b', alpha = 1)
    ax1.axes.set_xlabel('ave daily rainfall (mm): monthly (blue) yearly (grey)')
    
    #set up fill time as a datetime 64 object for matplotlib input
    #set up variables to plot the fill behind the yearly rain data
    filltime = (year_avg_sp.time.astype('datetime64'))
    zeros = np.zeros(shape=(len(filltime)))
    plt.fill_betweenx(filltime.time.values, zeros, year_avg_sp.rainfall, color ='k', alpha = 0.8)
    #set up variables to plot the fill behind the months data
    mzeros = np.zeros(shape=(len(month_sp.rainfall.time.values)))
    plt.fill_betweenx(month_sp.rainfall.time.values, mzeros, month_sp.rainfall, color ='c', alpha = 1)
    plt.axis([0, month_sp.rainfall.max(),lastyearhov , firstyearhov])
    

    ax2 = plt.subplot(gs[0, 1])
    hov_multi_ndvi_drop.plot(x='distance', y='time', yincrease = False, cmap = ndvi_cmap, norm = ndvi_norm)
    
    #to turn total suspended matter off, comment out the following row:
    
    hov_multi_tsm.plot(x='distance', y='time', yincrease = False, cmap = tss_cmap, norm = tss_norm)
    plt.axis([0, hov_multi_ndvi_drop.distance.max(), lastyearhov , firstyearhov])
    ax2.set_anchor("SE")

    plt.suptitle(transect_name+' : '+transect_description, fontsize ='24')
    #plt.show()
    #save the output files!
    plt.savefig(savepath+shape_name+'HovPlot'+'.png',bbox_inches='tight')

    print (savepath)
    
    #make a dictionary of the data we want to save
    vars2pickle = {'rain_sp':rain_sp, 
                   'month_sp':month_sp,
                   'hov_multi':hov_multi,
                   'hov_multi_ndvi_drop':hov_multi_ndvi_drop,
                   'hov_multi_tsm':hov_multi_tsm, 
                   #'ts_multi_ndvi':ts_multi_ndvi, 
                   'ndvi_cmap': ndvi_cmap, 
                   'ndvi_norm': ndvi_norm, 
                   'tss_cmap': tss_cmap, 
                   'tss_norm': tss_norm, 
                   'savepath': savepath, 
                   'affine':affine}

    #save the dictionary into a pickle file
    f = open(savepath+'/Hov_plot_variables'+'.pkl', 'wb')
    pickle.dump(vars2pickle,f) 
    f.close()
    print()
    # f=open(savepath+'/Hov_plot_variables'+'.pkl','rb')
    # saved_vars = pickle.load(f)
    # print(saved_vars.values())
    
    break

choose the index of the transect you would like to plot:0
Doongmabulla Springs 1 : Mouldy Crumpet to Joshua spring Nth Sth transect
Doongmabulla_Springs_1_ix0
OS error: [Errno 17] File exists: '/g/data/r78/rjd547/groundwater_activities/GalileeBasin/pickledata/Doongmabulla_Springs_1_ix0/'


ModuleNotFoundError: No module named 'pandas.tseries.index'